<a href="https://colab.research.google.com/github/hamzzihyun/competition/blob/main/%ED%92%8D%EB%A0%A5%EB%B0%9C%EC%A0%84%EB%9F%89%EC%98%88%EC%B8%A1%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/DACON/풍력 발전량 예측 AI/open.zip') as z:
  z.extractall()

## Import

In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
import random
import os
import numpy as np

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore')

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
train = TabularDataset('./train.csv')
test = TabularDataset('./test.csv')

In [ ]:
train.head()

,id,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,snowing,cloudiness,target
0,TRAIN_00000,27.8816,1013.6,46.8,2.4,232.4,0.00,False,0.0,1.180
1,TRAIN_00001,5.7540,1026.6,76.6,3.6,172.0,0.00,False,8.8,8.581
2,TRAIN_00002,20.8220,1016.2,64.8,2.2,206.0,0.06,False,23.0,3.978
3,TRAIN_00003,20.0758,1017.4,72.6,0.8,215.4,0.00,False,0.0,8.301
4,TRAIN_00004,7.5260,1023.2,82.8,1.2,158.0,0.00,False,15.0,1.692


## Feature & Target Split

In [ ]:
# # X는 독립변수이므로 종속변수를 제거합니다. 또한 target 이외의 분석에 활용하지 않는 데이터(id)를 제거합니다.
# train_x = train.drop(columns=['id', 'target'])
# # y는 종속변수로 값을 설정합니다.
# train_y = train['target']

In [ ]:
# # train에서와 마찬가지로 분석에 활용하지 않는 데이터(id)를 제거합니다.
# test_x = test.drop(columns=['id'])

## Data Pre-processing

In [ ]:
# le = LabelEncoder()
# le = le.fit(train_x['snowing'])
# train_x['snowing'] = le.transform(train_x['snowing'])

# for label in np.unique(test_x['snowing']):
#     if label not in le.classes_:
#         le.classes_ = np.append(le.classes_, label)
#     test_x['snowing'] = le.transform(test_x['snowing'])
# print('Done.')

## Modeling

### AutoGloun

In [ ]:
label = 'target'
eval_metric = 'mae'
time_limit = 3600 * 3

In [ ]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train, presets='best_quality', time_limit=time_limit)

No path specified. Models will be saved in: "AutogluonModels/ag-20230213_100849/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels/ag-20230213_100849/"
AutoGluon Version:  0.6.2
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    19275
Train Data Columns: 9
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (17.344, 0.0, 5.47703, 3.21758)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Usi

In [ ]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch_BAG_L2,-1.843160,783.926745,5272.403756,0.551535,158.929577,2,True,20
1,WeightedEnsemble_L3,-1.857165,798.930186,6624.558938,0.000565,0.343957,3,True,22
2,NeuralNetFastAI_BAG_L2,-1.860144,794.359628,6382.525422,10.984418,1269.051242,2,True,18
3,XGBoost_BAG_L2,-1.869832,785.771538,5312.767508,2.396329,199.293328,2,True,19
4,CatBoost_BAG_L2,-1.872512,784.999610,5485.574405,1.624401,372.100226,2,True,16
5,LightGBM_BAG_L2,-1.872905,787.537669,5293.393173,4.162460,179.918994,2,True,14
6,LightGBMXT_BAG_L2,-1.873086,793.187276,5311.955443,9.812066,198.481264,2,True,13
7,LightGBMLarge_BAG_L2,-1.874136,791.418003,5425.075471,8.042794,311.601292,2,True,21
8,WeightedEnsemble_L2,-1.884888,356.043670,884.679073,0.000626,0.413814,2,True,12
9,LightGBMLarge_BAG_L1,-1.885407,354.704746,872.805901,354.704746,872.805901,1,True,11


## Prediction

In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test, model=model_to_use)

## Submit

In [ ]:
# 제출 파일을 불러옵니다.
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['target'] = model_pred
submit.head()

,id,target
0,TEST_00000,4.799720
1,TEST_00001,5.131878
2,TEST_00002,8.052363
3,TEST_00003,4.554384
4,TEST_00004,4.720826


In [ ]:
submit.to_csv('./풍력발전량예측(2).csv', index=False)